In [121]:
from PIL import Image, ImageDraw, ImageFont
import math
import os

def PostProcess(inputfile, outputfile, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    with open(inputfile) as f:
        content = f.readlines()

    line = 1
    nodesCount = int(content[line])
    line += 1

    nodes = [(float(content[i].split()[0]), float(content[i].split()[1])) for i in range(line, line + nodesCount)]
    line += nodesCount
    
    min_x, min_y = max_x, max_y = nodes[0]
    for x, y in nodes:
        min_x, max_x = min(min_x, x), max(max_x, x)
        min_y, max_y = min(min_y, y), max(max_y, y)
    
    elementsCount = int(content[line])
    line += 1
    elements = [tuple(map(int, content[i].split())) for i in range(line, line + elementsCount)]
    
    center = [(max_x + min_x) / 2.0, (max_y + min_y) / 2.0]
    scale = max(max_x - min_x, max_y - min_y)
    
    with open(outputfile) as f:
        resultsContent = list(map(float, f.readlines()))
    
    deforms = resultsContent[:nodesCount * 2]
    stresses = [s for s in resultsContent[nodesCount * 2:]]  # Преобразуем в кН/м
    
    min_stress, max_stress = min(stresses), max(stresses)
    print(f"Max stress: {max_stress:.2f} MPa")
    
    def Transform(point):
        x = (point[0] - center[0]) / scale / 1.5 * image.size[0] + image.size[0] / 2.0
        y = (point[1] - center[1]) / scale / 1.5 * image.size[1] + image.size[1] / 2.0
        return (x, image.size[1] - y)
    
    image = Image.new("RGB", (1024, 1024))
    draw = ImageDraw.Draw(image)
    draw.rectangle([(0, 0), image.size], fill=0x777777)
    
    for element in elements:
        draw.polygon([Transform(nodes[element[i]]) for i in range(3)], fill=0x20a020, outline=0x707020)
    
    image.save(os.path.join(output_folder, "initial.png"), "PNG")
    draw.rectangle([(0, 0), image.size], fill=0x777777)
    
    if max_stress == min_stress:
        max_stress += 1.0
        min_stress -= 1.0
    
    for idx, element in enumerate(elements):
        v = (stresses[idx] - min_stress) / (max_stress - min_stress)
        colorf = ((v - 0.5) * 4.0, 2.0 - abs(v - 0.5) * 4.0, 2.0 - v * 4.0)
        colori = (int(colorf[0] * 255), int(colorf[1] * 255), int(colorf[2] * 255))
        draw.polygon([Transform((nodes[element[i]][0] + deforms[2 * element[i]] * 50.0,
                                 nodes[element[i]][1] + deforms[2 * element[i] + 1] * 50.0)) for i in range(3)], fill=colori)
    
    draw_legend(draw, image, min_stress, max_stress)
    image.save(os.path.join(output_folder, "sigmaMises.png"), "PNG")

def draw_legend(draw, image, min_stress, max_stress):
    legend_x = image.size[0] - 150
    legend_y = 100
    legend_height = 800
    steps = 50
    
    for i in range(steps):
        v = i / steps
        colorf = ((v - 0.5) * 4.0, 2.0 - abs(v - 0.5) * 4.0, 2.0 - v * 4.0)
        colori = (int(colorf[0] * 255), int(colorf[1] * 255), int(colorf[2] * 255))
        y_pos = legend_y + (1 - v) * legend_height
        draw.rectangle([(legend_x, y_pos), (legend_x + 30, y_pos + legend_height / steps)], fill=colori)
    
    font = ImageFont.truetype("arial.ttf", 20)  
    draw.text((legend_x, legend_y - 20), f"{max_stress:.2f} MPa", fill=(255, 255, 255), font=font)
    draw.text((legend_x, legend_y + legend_height + 5), f"{min_stress:.2f} MPa", fill=(255, 255, 255), font=font)


In [122]:
directory =  "C:/fydesis_cmake/result/"
inputfile = directory + "input_check.txt"  # Путь к файлу с узлами и элементами
outputfile = directory + "output.txt"  # Путь к файлу с деформациями и напряжениями
output_folder = directory  # Папка для сохранения изображений

PostProcess(inputfile, outputfile, output_folder)

Max stress: 300.85 MPa
